**An Intelligent Plant Disease Detection System Using CNN for Smart Hydroponics**

Author:[Aminu Musa] Date created: 18/08/2022
Description: This notebook is based on the paper an Intelligent Plant Disease Detection model Presented at 2021 IEEE 14th International Symposium on Embedded Multicore/Many-core Systems-on-Chip (MCSoC), Singapore.
The paper investigated the usefulness of CNN in plant disease detection in smart  hydroponics for control Agriculture. 
* We were able to achieve SOTA of 98% but the model is to big to fit on the embeded devices intended to use in the hydrpoinc system. the complete paper can be found at: https://ieeexplore.ieee.org/abstract/document/9691971 or 
DOI: 10.1109/MCSoC51149.2021.00058. 

* We used Tensorflow and Keras when impletmeting this work, in this particular notebook, due to limited computational resources we used smaller dataset with fewer instances than the original dataset we used in the paper. 

* In this notebook we provides detailed explanation on how Tensorflow can be used to implement CNN from Scratch.


we start with the importing all the required libraries in the code block below

In [ ]:
import numpy as np
import pickle
import cv2
import tensorflow as tf
from os import listdir
from sklearn.preprocessing import LabelBinarizer

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt

In the code block below, we initialize some important parameters needed to build a good model
1- we start with epochs which signifies the number of time the model can go over a training set iteratively.
2- BS stands for Batch Size, the nuber of images that can be fetch at a time
3- Default image size defines a unifrom size to all the images, because the model will expect to recieve images with uniform shapes as input.
4- lastly we define the height, width and depth of an image. 

In [ ]:
EPOCHS = 10
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
width=256
height=256
depth=3
directory_root = "../input/plantdisease"

Here we used the built in keras function img_to_array to preprocess our images and convert them into numpy array 

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

in code block below we tried to fetch all the images and thier corresponsing labels from dataset directory and saved them in an image list And labels list respectively. in each sub directory we check to see if there is DS file and remove it. We fetch anything that ends with .jpg signifying that its an image file, alongside the corresponding labels.

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)
            
    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)
        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)
            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
    
        
    
    
except Exception as e:
    print(f"Error : {e}")

the label binarizer we imported from SK learn helps transform image Labels  into numpy array, which can be predicted by the model

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print (n_classes)

Print the classes

Next thing, is to tranform the image list into numpy array for easier manipulation, and normalized the values by dividing through by 225

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

Another sk learn package train_test_split was used here to divide the dataset into two separate sets, normally called training and testing sets. 

In [ ]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)


We used tesnorflow package Imagedatagenerator to augment our dataset for better generalization and performance, Note: Augmentation here refers to the process of changing the normal image attributes such as rotating to diffent angle zoom in, to create another imaginary images that are not present in the dataset.


In [ ]:
'''
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
'''

The Model architecture is define below:  it takes images as input with uniform shape as defined in the beginging of the notebook, followed by 5 convolutional layer, each with a relu activation function. Then a drop out layer was added and and flattening layer before final dense layer with n_classes number of neurons and a softmax activation fun ioin to obtain the predicted probabilities.

In [ ]:

model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

Model Summary

The function model.summary() print the tabular form of the model architecture with the number of parameters available in the model

In [ ]:
model.summary()

Model.compile applies loss function and an optimizer to the model, allowing the learning process and optimiztion to take place.

In [ ]:

model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.0001),metrics=["accuracy"])



Model.fit() commences the traning of the model saving the accuracy for each epoch in the history variable.

In [ ]:
history = model.fit(
    x_train, y_train, batch_size=BS,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Plot the train and val curve

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print(range(1, len(acc) + 1))

epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()



plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

Print the model Accuracy

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

Save the model using Pickle

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))''